<a href="https://colab.research.google.com/github/ClarinetInALeatherJacket/OgataLabROP2024/blob/main/EISAnalysis_Ruben'sVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Documentation**

https://docs.google.com/document/d/1Y02BeM-ZGgUQ62iDPRuGmKf828ejeg-cdS43H4aq1B0/edit?usp=sharing


**Import Statements**

In [ ]:
import os
import shutil
import codecs
import time
#!pip install numpy
import numpy as np
#!pip install pandas
import pandas as pd
#!pip install matplotlib
import matplotlib.pyplot as plt
from google.colab import files
%load_ext google.colab.data_table
#!pip install ipdb
#import ipdb
#%pdb off

**Global Variables and Classes**

In [ ]:
# Store the names of the columns in an array for easy access
columns = ["freq / Hz", "Z' / Ohm", "Z'' / Ohm", "None"]

# Store possible graph line parameters in a list, to be used on graphs w/
# multiple lines
plot_lines = ['o-b', '^-r', 's-g', 'D-k', 'o-y', '^-m', 's-c']


# Custom class to store the data. Stores a name, code, and any data, and
# automatically generates the mean and standard deviations
class Experiment:
    code: str
    name: str
    data = []
    mean = pd.DataFrame
    std = pd.DataFrame

    def __init__(self, code: str, name: str):
        self.code = code
        self.name = name
        self.data = []
        self.mean = pd.DataFrame
        self.std = pd.DataFrame

    def add_data(self, data):
        self.data.append(data)
        # Any time new data is added, update the mean and deviation. If there is
        # only one trial and the deviation comes up as NANs, fill w/ 0
        self.mean = pd.concat(self.data).groupby("freq / Hz").mean()
        self.std = pd.concat(self.data).groupby("freq / Hz").std()
        self.std = self.std.fillna(value=0)


**Function to Clear Temporary Input and Converted Directories**

In [ ]:
def clear_temp_dirs():
  """
  Empties and deletes the ./Input and ./Converted directories created during
  parse_input().
  """
  if os.path.exists("./Input"):
    for file in os.listdir("./Input"):
      os.remove(f"./Input/{file}")
    os.rmdir("./Input")

  if os.path.exists("./Converted"):
    for file in os.listdir("./Converted"):
      os.remove(f"./Converted/{file}")
    os.rmdir("./Converted")

  if os.path.exists("./Output"):
    for file in os.listdir("./Output"):
      os.remove(f"./Output/{file}")
    os.rmdir("./Output")


**Function to Parse Input File**

In [ ]:
def parse_input()->list[Experiment]:
  """
  Takes in a single CSV file with multiple experiments and returns a list of
  experiment data structures, containing the numeric data and ID codes
  """
  # Get the file name for the inputted file
  input_file = list(files.upload().keys())[0]

  # If the Input and Converted directories don't exist, create them
  if not os.path.exists("./Input"):
    os.mkdir("./Input")
  if not os.path.exists("./Converted"):
    os.mkdir("./Converted")
  if not os.path.exists("./Output"):
    os.mkdir("./Output")

  # Move the input file to the input directory
  shutil.move(f"./{input_file}",f"./Input/{input_file}")

  # Convert input file from UTF16 LE BOM encoding to UTF8
  with codecs.open(f"./Input/{input_file}", "r", encoding="utf-16") as f:
    with codecs.open(f"./Converted/{input_file}", "w",encoding="utf-8") as f2:
      shutil.copyfileobj(f, f2)

  output = []

  # Read the input file and turn it into an array of lines
  with open(f"./Converted/{input_file}") as file:
      lines = file.read().splitlines()

  # Start a counter to iterate through all the input file lines and a set to
  # track which codes have been encountered before
  i = 0
  seen_codes = set()


  while i < len(lines) - 1:

    # If a line starting with "Measurement", in other words the beginning of
    # a data set header, is found, begin processing the data set
    if lines[i].startswith("Measurement"):

      # Extract the name and code from the "Measurement" line
      split = lines[i].split(",")[1].split(".")
      code = split[0]
      name = split[1]

      # If the code has not been encountered before, create an experiment for
      # that code
      if code not in seen_codes:
        x = Experiment(code, name)
        output.append(x)
        seen_codes.add(code)

      # Locate the index of the line that immediately precedes the numeric data
      while not lines[i].startswith("freq"):
        i += 1

      start = i

      # Locate the index of the empty line that immediately succeeds the numeric
      # data
      while i < len(lines) - 1 and not lines[i + 1] == "":
        i += 1

      end = i

      # Locate the experiment that coincides with this numeric data's code, then
      # add the data to that experiment
      for j in output:
        if j.code == code:
          temp = pd.read_csv(f"./Converted/{input_file}",
                              names=columns,
                              skiprows=start + 1,
                              nrows = end - start)
          temp = temp.dropna(axis=1) # Drop an extra column that is formed
                                     # due to CSV file format
          j.add_data(temp)

      i += 1

    else:
      i += 1

  return output

**Function to plot all graphs**

In [ ]:
def create_plots(data: list[list[Experiment]]):

  """
  Takes in a list of lists of experiments and plots them via subfunctions
  accoring to patterns detailed in the documentations. Provides updates when
  each subfunction has been completed.
  """

  plot_single_nyquists(data)
  print("• Single experiment Nyquists successfully plotted \n")

  plot_group_nyquists(data)
  print("• Complete set Nyquist data successfully plotted \n")

  plot_as_synthesized_deltas(data)
  print("• Delta relative to As Synthesized successfully plotted \n")

  plot_other_deltas(data)
  print("• Remaining Deltas successfully plotted \n")

  return None

**Function to Plot all Single Data Nyquist Plots**

In [ ]:
def plot_single_nyquists(data: list[list[Experiment]]):

  """
  Takes in all experiments and creates a Nyquist plot for each one. Includes
  standard dev as error. Saves to the 'Output' folder
  """

  plt.figure("singles")

  for i in data:
    for j in i:
      plt.errorbar(j.mean["Z' / Ohm"], j.mean["Z'' / Ohm"],
                   xerr=j.std["Z' / Ohm"], yerr=j.std["Z'' / Ohm"],
                   fmt='o-b', label=j.name, linewidth=0.5)
      plt.title(j.name)
      plt.xlabel("Z' (Ohm)")
      plt.ylabel("Z'' (Ohm)")
      plt.savefig(f"./Output/{j.name} Nyquist.png",
                  bbox_inches="tight", pad_inches=0.5)
      plt.clf()


**Function to Plot all Multiple Data Nyquist Plots**

In [ ]:
def plot_group_nyquists(data: list[list[Experiment]]):

    """
    Takes in all experiments and creates Nyquist plots with multiple experiments.
    All experiements with codes 0xx, 1xx, and 2xx, are plotted onto one graph.
    After that, all groups with the same first digit of their code are plotted
    together. All groups whose code begin with odd numbers also plot the last
    experiment of the previous groupe. For example, the group with the codes 3xx
    will be plotted along with the experiment with the highest code beginning with
    2. See official documentation for more detail.
    """

    # plot the custom graph including 0xx, 1xx, and 2xx code experiments
    plt.figure("group")

    plt.errorbar(data[0][0].mean["Z' / Ohm"], data[0][0].mean["Z'' / Ohm"],
                 xerr=data[0][0].std["Z' / Ohm"],
                 yerr=data[0][0].std["Z'' / Ohm"],
                 fmt='o-b', label=data[0][0].name, linewidth=0.5)
    plt.errorbar(data[1][0].mean["Z' / Ohm"], data[1][0].mean["Z'' / Ohm"],
                 xerr=data[1][0].std["Z' / Ohm"],
                 yerr=data[1][0].std["Z'' / Ohm"],
                 fmt='^-r', label=data[1][0].name, linewidth=0.5)
    plt.errorbar(data[2][0].mean["Z' / Ohm"], data[2][0].mean["Z'' / Ohm"],
                 xerr=data[2][0].std["Z' / Ohm"],
                 yerr=data[2][0].std["Z'' / Ohm"],
                 fmt='s-g', label=data[2][0].name, linewidth=0.5)
    plt.xlabel("Z' (Ohm)")
    plt.ylabel("Z'' (Ohm)")
    plt.title("Synthesis")
    plt.legend()
    plt.savefig(f"./Output/Synthesis Nyquist.png", bbox_inches="tight",
                pad_inches=0.5)
    plt.clf()


    # Cycle through the rest of the experiements, plotting them together
    # according to their codes
    name_list = []
    line_num = 0
    for i in range(2, len(data)):

        if len(data[i]) == 0:
            continue

        # If the code begins with an odd digit, add the last experiment from the
        # previous code to the graph
        if i % 2 != 0:
            plt.errorbar(data[i - 1][-1].mean["Z' / Ohm"],
                         data[i - 1][-1].mean["Z'' / Ohm"],
                         xerr=data[i - 1][-1].std["Z' / Ohm"],
                         yerr=data[i - 1][-1].std["Z'' / Ohm"],
                         fmt=plot_lines[line_num % len(plot_lines)],
                         label=data[i - 1][-1].name,
                         linewidth=0.5)
            line_num += 1

        # plot all experiments. Add details to graph. Save and resest graph
        for j in data[i]:
            plt.errorbar(j.mean["Z' / Ohm"], j.mean["Z'' / Ohm"],
                         xerr=j.std["Z' / Ohm"],
                         yerr=j.std["Z'' / Ohm"],
                         fmt=plot_lines[line_num % len(plot_lines)],
                         label=j.name,
                         linewidth=0.5)
            name_list.append(j.name)
            line_num += 1

        name = os.path.commonprefix(name_list).replace("(", "").replace(")", "")
        plt.title(name)
        plt.xlabel("Z' (Ohm)")
        plt.ylabel("Z'' (Ohm)")
        plt.legend()
        plt.savefig(f"./Output/{name} Nyquist.png", bbox_inches="tight",
                    pad_inches=0.5)
        plt.clf()

**Function to Plot the Delta Graphs Relative to As Synthesized**

In [ ]:
def plot_as_synthesized_deltas(data: list[list[Experiment]]):

    """
    Takes in all experiments and plots the deltas that are measured relative to
    the "as synthesized" experiment, code 000. The experiments plotted are the 1xx
    and 2xx experiments.
    """


    # Complicated one liners essentially just get the experiments and their
    # delta references, take the differences of the data, and group them by the
    # frequency column, using it as an index. The mean and standard dev of these
    # groups is taken, along with the log base 10 of the frequency for the delta
    # plots
    Group1 = pd.concat([data[1][0].data[0].set_index("freq / Hz") -
                        data[0][0].data[0].set_index("freq / Hz"),
                        data[1][0].data[1].set_index("freq / Hz") -
                        data[0][0].data[1].set_index("freq / Hz")]).groupby(level=0)

    delta_avg_1 = Group1.mean()
    delta_avg_1["freq / Hz"] = np.log10(delta_avg_1.index)

    delta_dev_1 = Group1.std()


    Group2 = pd.concat([data[2][0].data[0].set_index("freq / Hz") -
                        data[0][0].data[0].set_index("freq / Hz"),
                        data[2][0].data[1].set_index("freq / Hz") -
                        data[0][0].data[1].set_index("freq / Hz")]).groupby(level=0)

    delta_avg_2 = Group2.mean()
    delta_avg_2["freq / Hz"] = np.log10(delta_avg_2.index)

    delta_dev_2 = Group2.std()


    # Plots and saves the delta graphs to the 'Output' folder
    plt.clf()
    plt.errorbar(delta_avg_1["freq / Hz"], delta_avg_1["Z' / Ohm"],
                 yerr=delta_dev_1["Z' / Ohm"], fmt="o-b", label=data[1][0].name,
                 linewidth=0.5)
    plt.errorbar(delta_avg_2["freq / Hz"], delta_avg_2["Z'' / Ohm"],
                 yerr=delta_dev_2["Z'' / Ohm"], fmt="^-r",
                 label=data[2][0].name,
                 linewidth=0.5)
    plt.axhline(0, color='black', lw=0.5)
    plt.title("Δ Z'")
    plt.xlabel("log(frequency) (Hz)")
    plt.ylabel("Δ Z' (Ohm)")
    plt.legend()
    plt.savefig("./Output/Delta Z' relative to As Synthesized.png", bbox_inches="tight",
                pad_inches=0.5)
    plt.clf()

    plt.errorbar(delta_avg_1["freq / Hz"], delta_avg_1["Z'' / Ohm"],
                 yerr=delta_dev_1["Z'' / Ohm"], fmt="o-b",
                 label=data[1][0].name,
                 linewidth=0.5)
    plt.errorbar(delta_avg_2["freq / Hz"], delta_avg_2["Z'' / Ohm"],
                 yerr=delta_dev_2["Z'' / Ohm"], fmt="^-r",
                 label=data[2][0].name,
                 linewidth=0.5)
    plt.axhline(0, color='black', lw=0.5)
    plt.title("Δ Z''")
    plt.xlabel("log(frequency) (Hz)")
    plt.ylabel("Δ Z'' (Ohm)")
    plt.legend()
    plt.savefig("./Output/Delta Z'' relative to As Synthesized.png", bbox_inches="tight",
                pad_inches=0.5)
    plt.clf()

**Function to Plot the remaining Delta Plots**

In [ ]:
def plot_other_deltas(data: list[list[Experiment]]):

    """
    Takes in all the experiments and plots the delta graphs of experiments in the
    200 code range and above, all the way to 900. Plots group graphs similar to
    plot_group_nyqiusts. For all experiments, if the first digit of the code is
    even, they use the first item in their set as the standard. If they are odd,
    they use the last item of the previous set. For example, [4][2] is measured
    relative to [4][0], and [5][3] is measured relative to [4][-1]
    """

    plt.figure("delta Z'")
    plt.figure("delta Z''")

    for i in range(2, 9):

        line_num = 0
        plt.clf()

        if len(data[i]) == 0:
            continue

        # set the standard according to documentation rules
        if i % 2 == 0:
            standard = data[i][0]
        else:
            standard = data[i - 1][-1]

        for j in range(len(data[i])):

            if data[i][j] == standard:
                continue

            # Check if the experiment and the standard have the same number of
            # trials. If so, calcuate the delta of each trial and use the mean
            # and standard dev. If not, calculate the delta relative to the mean
            # of the standard and assume standard dev = 0
            if len(data[i][j].data) == len(standard.data):
                Groupby = ((pd.concat([data[i][j].data[x].set_index("freq / Hz")
                                       - standard.data[x].set_index("freq / Hz")
                                       for x in range(len(data[i][j].data))]))
                           .groupby(level=0))

            else:
                Groupby = (pd.concat([x.set_index("freq / Hz") -
                                      standard.mean
                                      for x in data[i][j].data])
                           .groupby(level=0))

            delta_avg = Groupby.mean()
            delta_avg["freq / Hz"] = np.log10(delta_avg.index)

            delta_dev = Groupby.std()
            delta_dev = delta_dev.fillna(value=0)

            # add plots to graphs
            plt.figure("delta Z'")
            plt.errorbar(delta_avg["freq / Hz"], delta_avg["Z' / Ohm"],
                         yerr=delta_dev["Z' / Ohm"], fmt=plot_lines[line_num % len(plot_lines)],
                         label=data[i][j].name, linewidth=0.5)

            plt.figure("delta Z''")
            plt.errorbar(delta_avg["freq / Hz"], delta_avg["Z'' / Ohm"],
                         yerr=delta_dev["Z'' / Ohm"], fmt=plot_lines[line_num % len(plot_lines)],
                         label=data[i][j].name, linewidth=0.5)

            line_num += 1

        # add details to graphs, save and reset graphs
        name = standard.name
        plt.figure("delta Z'")
        plt.axhline(0, color='black', lw=0.5)
        plt.title(f"Δ Z' relative to {name}")
        plt.xlabel("log(frequency) (Hz)")
        plt.ylabel("Δ Z' (Ohm)")
        plt.legend()
        plt.savefig(f"./Output/Delta Z' relative to {name}.png",
                    bbox_inches="tight",
                    pad_inches=0.5)
        plt.clf()
        plt.figure("delta Z''")
        plt.axhline(0, color='black', lw=0.5)
        plt.title(f"Δ Z'' relative to {name}")
        plt.xlabel("log(frequency) (Hz)")
        plt.ylabel("Δ Z'' (Ohm)")
        plt.legend()
        plt.savefig(f"./Output/Delta Z'' relative to {name}.png",
                    bbox_inches="tight",
                    pad_inches=0.5)
        plt.clf()

**Main Code**

In [ ]:
# Ensure that directories used by the code are set up as expected
clear_temp_dirs()

# Take in the CSV file, parse it, form it into experiments, and the return those
# experiments
experiments = parse_input()

print("• All data successfully parsed \n")

# Organize the experiments by their codes: The first digit determines their
# column, the last two determine their row
organized_experiments = [[], [], [], [], [], [], [], [], []]
for x in experiments:
  organized_experiments[int(x.code[0])].append(x)

for x in organized_experiments:
  x.sort(key=lambda i: i.code)

# Plot all the required graphs, save them to the Output folder
create_plots(organized_experiments)

# Compress the Output folder to a zip file and download it
!zip -r EISAnalysisOutput.zip ./Output

files.download("./EISAnalysisOutput.zip")

clear_temp_dirs()